In [2]:
import tensorflow.compat.v1 as tf
tf.enable_eager_execution()


/Users/chanzy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
dataset = input_fn_builder('/Users/chanzy/Desktop/tydiqa/tydiqa/baseline/train_samples.tfrecord', 512, True, False)

Instructions for updating:
Use `tf.cast` instead.


In [2]:
def input_fn_builder(input_file, seq_length, is_training, drop_remainder):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""

    # This needs to be kept in sync with `FeatureWriter`.
    name_to_features = {
      "language_id": tf.FixedLenFeature([], tf.int64),
      "unique_ids": tf.FixedLenFeature([], tf.int64),
      "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
      "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
    }

    if is_training:
        name_to_features["start_positions"] = tf.FixedLenFeature([], tf.int64)
        name_to_features["end_positions"] = tf.FixedLenFeature([], tf.int64)
        name_to_features["answer_types"] = tf.FixedLenFeature([], tf.int64)

    def _decode_record(record, name_to_features):
        example = tf.parse_single_example(record, name_to_features)

        # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
        # So cast all int64 to int32.
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.to_int32(t)
                example[name] = t

        return example


    # For training, we want a lot of parallel reading and shuffling.
    # For eval, we want no shuffling and parallel reading doesn't matter.
    d = tf.data.TFRecordDataset(input_file)
    d = d.shuffle(buffer_size=100)
    d = d.map(lambda record: _decode_record(record, name_to_features))
    return d

In [4]:
def dataset_fn(ds, i): 
    return ds.filter(lambda x: x['language_id']==tf.to_int32(i))
data_set_lst = []
for i in range(11):
    dataset_filter_lang = dataset.apply(lambda x: dataset_fn(x, i))
    data_set_lst.append(dataset_filter_lang)

In [5]:
def split_train_dev(dataset, train_port, num_dev):
    train_size = int(DATASET_SIZE * train_port)
    sub_dev_size = int(DATASET_SIZE * (1-train_port)/num_dev)
    
    full_dataset = dataset.shuffle(100)
    train_dataset = full_dataset.take(train_size)
    test_dataset = full_dataset.skip(train_size)
    
    shard_devs = []
    for i in range(num_dev):
        shard_devs.append(test_dataset.shard(num_shards=num_dev, index=i))
    return train_dataset, shard_devs
        
    

In [6]:
DATASET_SIZE = 41
train_dataset, dev_shards = split_train_dev(dataset, 0.5, 2)

In [17]:
next(iter(train_dataset))['language_id']

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [7]:
def count_dataset(dataset):
    cnt = 0
    for i in dataset:
        #if cnt % 2000==0:
            #print(cnt)
        cnt += 1
    
    return cnt
count_dataset(train_dataset)

20

In [8]:
count_dataset(dev_shards[1])

10

In [9]:
def split_langs(dataset):
    def dataset_fn(ds, i):
        return ds.filter(lambda x: x['language_id'] == i)
    data_set_lst = []
    for i in range(12):
        dataset_filter_lang = dataset.apply(lambda x: dataset_fn(x, i))
        data_set_lst.append(dataset_filter_lang)
    return data_set_lst

In [10]:
train_set_langs = split_langs(train_dataset)

In [11]:
count_dataset(train_set_langs[2])

4

In [12]:
for i in train_set_langs:
    print(count_dataset(i))

2
0
1
0
0
0
2
2
3
2
3
0


In [31]:
dev_set_langs = []
for div_set in dev_shards:
    dev_set_langs.append(split_langs(div_set))

In [36]:
train_set_sampler = list(map(lambda x: iter(x.repeat()), train_set_langs))

In [37]:
dev_set_sampler = []
for dev_set in dev_set_langs:
    dev_set_sampler.append(list(map(lambda x: iter(x.repeat()), dev_set)))

<tf.Tensor: shape=(), dtype=int32, numpy=-1085280860>

In [ ]:
global_step = tf.train.get_or_create_global_step()

  learning_rate = tf.constant(value=init_lr, shape=[], dtype=tf.float32)

  # Implements linear decay of the learning rate.
  learning_rate = tf.train.polynomial_decay(
      learning_rate,
      global_step,
      num_train_steps,
      end_learning_rate=0.0,
      power=1.0,
      cycle=False)

  # Implements linear warmup. I.e., if global_step < num_warmup_steps, the
  # learning rate will be `global_step/num_warmup_steps * init_lr`.
  if num_warmup_steps:
    global_steps_int = tf.cast(global_step, tf.int32)
    warmup_steps_int = tf.constant(num_warmup_steps, dtype=tf.int32)

    global_steps_float = tf.cast(global_steps_int, tf.float32)
    warmup_steps_float = tf.cast(warmup_steps_int, tf.float32)

    warmup_percent_done = global_steps_float / warmup_steps_float
    warmup_learning_rate = init_lr * warmup_percent_done

    is_warmup = tf.cast(global_steps_int < warmup_steps_int, tf.float32)
    learning_rate = (
        (1.0 - is_warmup) * learning_rate + is_warmup * warmup_learning_rate)

  # It is recommended that you use this optimizer for fine tuning, since this
  # is how the model was trained (note that the Adam m/v variables are NOT
  # loaded from init_checkpoint.)
  optimizer = AdamWeightDecayOptimizer(
      learning_rate=learning_rate,
      weight_decay_rate=0.01,
      beta_1=0.9,
      beta_2=0.999,
      epsilon=1e-6,
      exclude_from_weight_decay=["LayerNorm", "layer_norm", "bias"])

  if use_tpu:
    optimizer = contrib_tpu.CrossShardOptimizer(optimizer)

  tvars = tf.trainable_variables()
  grads = tf.gradients(loss, tvars)

  # This is how the model was pre-trained.
  (grads, _) = tf.clip_by_global_norm(grads, clip_norm=1.0)

  train_op = optimizer.apply_gradients(
      zip(grads, tvars), global_step=global_step)

  # Normally the global step update is done inside of `apply_gradients`.
  # However, `AdamWeightDecayOptimizer` doesn't do this. But if you use
  # a different optimizer, you should probably take this line out.
  new_global_step = global_step + 1
  train_op = tf.group(train_op, [global_step.assign(new_global_step)])
  return train_op

In [48]:
phi = tf.get_variable(
"phi", [10], initializer=tf.truncated_normal_initializer(stddev=0.02))

In [49]:
loss = tf.log(tf.nn.softmax(phi))[0]

In [14]:
loss = tf.log(tf.nn.softmax(phi))

In [20]:
list(tf.nn.softmax(phi).numpy())

[0.1017699,
 0.099157505,
 0.100775644,
 0.09762852,
 0.09632411,
 0.10006377,
 0.10062185,
 0.10204338,
 0.09812517,
 0.10349009]

In [50]:
with tf.GradientTape() as tape:
    loss = tf.log(tf.nn.softmax(phi))
grads = tape.gradient(loss, phi)
with tf.GradientTape() as tape:
    loss = tf.log(tf.nn.softmax(phi))
grads2 = tape.gradient(loss, phi)

In [54]:
normalize_a = tf.nn.l2_normalize(grads,0)        
normalize_b = tf.nn.l2_normalize(grads2,0)
cos_similarity=tf.reduce_sum(tf.multiply(normalize_a,normalize_b))

In [55]:
cos_similarity

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [57]:
cos_similarity * grads

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([-0.00880536, -0.04625217,  0.01688722, -0.00595451,  0.0193012 ,
       -0.00927766,  0.01762019,  0.01860519,  0.02450252, -0.02662662],
      dtype=float32)>

In [42]:
with tf.GradientTape() as tape:
    loss = tf.log(tf.nn.softmax(phi))
grads2 = tape.gradient(tf.nn.softmax(phi)[0], phi)

In [47]:
tf.math.cos(grads, grads2)

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([0.99984336, 0.99996454, 0.9999699 , 0.99971884, 0.99932444,
       0.9999998 , 0.9999807 , 0.99979126, 0.9998242 , 0.999391  ],
      dtype=float32)>

In [36]:
print(grads2)

None


In [3]:

global_step = tf.train.get_or_create_global_step()

In [9]:
if global_step == 0:
    print('fase')

fase


In [12]:
opt = tf.train.AdamOptimizer(
    learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False,
    name='Adam'
    )

In [58]:
tf.trainable_variables()

[]